Importing necessary modules from standard library

In [1]:
from collections import Counter

User error for the possible cases with modular equations in the form ax = b (mod n) :

Is used mainly in 2 cases:
    1. Inverse of a mod m doesn`t exist (so equation doesn`t have any solutions)
    2. GCD(a, n) = d (d !=1) and (b % d) != 0 (so equation doesn`t have any solutions)

In [2]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class ModuloError(Error):
    """Exception raised for errors in the input.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

Constants for lab:

Alphabet for folder 'variants' has swapped letters 'ь' and 'ы'. For folder 'for_test' unchanged russian alphabet is used

In [3]:
#classic russian alphabet without ъ
alphabet='абвгдежзийклмнопрстуфхцчшщыьэюя'
#replace ы and ь
alphabet='абвгдежзийклмнопрстуфхцчшщьыэюя'

In [4]:
m = len(alphabet)
m

31

Ranges is used when we have to combine top bigrams from russian language and ciphertext to find all possible keys. Ranges represent all possible combinations.

In [5]:
ranges = ((0,1),(0,2),(0,3),(0,4),(1,0), (1,2), (1,3), (1,4), (2,0), (2,1), (2,3), (2,4), (3,0), (3,1),
          (3,2), (3,4), (4,0), (4,1), (4,2), (4,3))

The most common bigrams in russian language

In [6]:
top_ngrams_rus = ['ст', 'но', 'то', 'на', 'ен']

Opening text of variant 13 as global variable, getting rid of newline symbol

In [7]:
text = open('variants/13.txt', 'r').read()
text = text.replace('\n', '')
text[:100]

'дюэорэдюэорэтнтфоэлкшэунскынайцбюовыюежэмюшыафткьэапжнечсюкэфэгябаейпблрщкбсяфийпкдчаясюлюхэитрэшуаф'

Basic mathematical functions for equation ax = b (mod m )

In [8]:
def egcd(a, b):
    '''ax + by = g = gcd(a, b)
       returns (g, x, y)'''
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
    

In [9]:
def modinv(a, n):
    '''Finding inverse of a mod n'''
    g, x, y = egcd(a, n)
    if abs(g) != 1:
        raise ModuloError("Inverse doesn't exist")
    else:
        return x % n

In [14]:
modinv(13, 18)

7

In [15]:
def equation_solver(a, b, n=m**2):
    ''' Solving equation ax = b (mod n) '''
    d = egcd(a, n)[0]
    if d == 1:
        return [(modinv(a, n) * b) % (n)]
    elif (b % d) != 0:
        raise ModuloError('No solutions')
    else: 
        a1 = a / d
        b1 = b / d
        n1 = n / d
        x0 = (b1 * modinv(a1, n1)) % n1
        return [ (x0+i*n1) for i in range(d)]
        
    

Several tests to check if equation_solver works properly

In [16]:
equation_solver(10, 11, 18)

ModuloError: No solutions

In [17]:
equation_solver(5, 13, 37)

[10]

In [12]:
equation_solver(1215, 560, 2755)

[200.0, 751.0, 1302.0, 1853.0, 2404.0]




Counting all bigrams in ciphertext

In [13]:
def ngram_counter(text, return_freq=True):
    '''Counts ngrams number/frequency'''
    counter = dict()
    
    data = [text[i:i+2] for i in range(0,(len(text)-2), 2)]
    for gram in data:
        if gram in counter:
            counter[gram]+=1
        if gram not in counter:
            counter[gram] = 1
    
    if return_freq:
        n_sum = sum(counter.values())
        for key,val in counter.items():
            counter[key] = val/n_sum
            
    
    return counter

In [14]:
num_ngrams = ngram_counter(text, return_freq=True)
print(num_ngrams)

{'дю': 0.012268188302425107, 'эо': 0.005135520684736091, 'рэ': 0.0019971469329529245, 'тн': 0.003708987161198288, 'тф': 0.006562054208273894, 'оэ': 0.003138373751783167, 'лк': 0.0002853067047075606, 'шэ': 0.003708987161198288, 'ун': 0.007132667617689016, 'ск': 0.0028530670470756064, 'ын': 0.010556348074179744, 'ай': 0.0005706134094151213, 'цб': 0.003138373751783167, 'юо': 0.005420827389443652, 'вы': 0.007703281027104137, 'юе': 0.0028530670470756064, 'жэ': 0.0019971469329529245, 'мю': 0.007988587731811698, 'шы': 0.003708987161198288, 'аф': 0.015121255349500713, 'тк': 0.005135520684736091, 'ьэ': 0.005991440798858773, 'ап': 0.011697574893009986, 'жн': 0.0094151212553495, 'еч': 0.0011412268188302425, 'сю': 0.007988587731811698, 'кэ': 0.010271041369472182, 'фэ': 0.00884450784593438, 'гя': 0.0005706134094151213, 'ба': 0.0005706134094151213, 'ей': 0.0011412268188302425, 'пб': 0.0017118402282453639, 'лр': 0.0017118402282453639, 'щк': 0.0048502139800285305, 'бс': 0.0014265335235378032, 'яф': 0.

Finding 5 most frequent bigrams in ciphertext

In [15]:
top_ngrams = sorted(num_ngrams.items(), key = lambda x: x[1], reverse=True)[:5]
print(top_ngrams)


[('аф', 0.015121255349500713), ('яф', 0.015121255349500713), ('дю', 0.012268188302425107), ('ап', 0.011697574893009986), ('нф', 0.011697574893009986)]


Unpacking bigrams

In [16]:
top_ngrams = [x[0] for x in top_ngrams]
top_ngrams

['аф', 'яф', 'дю', 'ап', 'нф']

Turning bigrams into numbers

In [17]:
def ngram_code(ngram): 
    Xi = alphabet.find(ngram[0])*m+alphabet.find(ngram[1])
    return Xi

Dicrionary of shape {ngram_code : ngram}

In [18]:
big_dict = {}
for i in alphabet:
    for j in alphabet:
        big_dict[i+j] = ngram_code(i+j)

In [19]:
big_dict = dict((v,k) for k,v in big_dict.items())
big_dict

{0: 'аа',
 1: 'аб',
 2: 'ав',
 3: 'аг',
 4: 'ад',
 5: 'ае',
 6: 'аж',
 7: 'аз',
 8: 'аи',
 9: 'ай',
 10: 'ак',
 11: 'ал',
 12: 'ам',
 13: 'ан',
 14: 'ао',
 15: 'ап',
 16: 'ар',
 17: 'ас',
 18: 'ат',
 19: 'ау',
 20: 'аф',
 21: 'ах',
 22: 'ац',
 23: 'ач',
 24: 'аш',
 25: 'ащ',
 26: 'аь',
 27: 'аы',
 28: 'аэ',
 29: 'аю',
 30: 'ая',
 31: 'ба',
 32: 'бб',
 33: 'бв',
 34: 'бг',
 35: 'бд',
 36: 'бе',
 37: 'бж',
 38: 'бз',
 39: 'би',
 40: 'бй',
 41: 'бк',
 42: 'бл',
 43: 'бм',
 44: 'бн',
 45: 'бо',
 46: 'бп',
 47: 'бр',
 48: 'бс',
 49: 'бт',
 50: 'бу',
 51: 'бф',
 52: 'бх',
 53: 'бц',
 54: 'бч',
 55: 'бш',
 56: 'бщ',
 57: 'бь',
 58: 'бы',
 59: 'бэ',
 60: 'бю',
 61: 'бя',
 62: 'ва',
 63: 'вб',
 64: 'вв',
 65: 'вг',
 66: 'вд',
 67: 'ве',
 68: 'вж',
 69: 'вз',
 70: 'ви',
 71: 'вй',
 72: 'вк',
 73: 'вл',
 74: 'вм',
 75: 'вн',
 76: 'во',
 77: 'вп',
 78: 'вр',
 79: 'вс',
 80: 'вт',
 81: 'ву',
 82: 'вф',
 83: 'вх',
 84: 'вц',
 85: 'вч',
 86: 'вш',
 87: 'вщ',
 88: 'вь',
 89: 'вы',
 90: 'вэ',
 91: 'вю'



Functions to turn plain text into ciphertext using key (a, b).
Not needed for lab but was written for testing reasons.

In [20]:
def encode(string, a , b):
    '''Turns one bigram of plaintext to ciphertext'''
    return (a*ngram_code(string)+b) % (m**2)

In [21]:
def encoder(string, a, b):
    '''Turns all plaintext to ciphertext'''
    res = ''
    data = [string[i:i+2] for i in range(0,(len(string)-2),2)]
    for bigram in data:
        res += big_dict[encode(bigram, a, b)]
    return res

Test

In [22]:
ngram_code('ст')

545

In [23]:
encode('ст', 2, 3)

132

In [24]:
ngram_code('но')

417

In [25]:
encode('но', 2, 3)

837

In [26]:
big_dict.items()

dict_items([(0, 'аа'), (1, 'аб'), (2, 'ав'), (3, 'аг'), (4, 'ад'), (5, 'ае'), (6, 'аж'), (7, 'аз'), (8, 'аи'), (9, 'ай'), (10, 'ак'), (11, 'ал'), (12, 'ам'), (13, 'ан'), (14, 'ао'), (15, 'ап'), (16, 'ар'), (17, 'ас'), (18, 'ат'), (19, 'ау'), (20, 'аф'), (21, 'ах'), (22, 'ац'), (23, 'ач'), (24, 'аш'), (25, 'ащ'), (26, 'аь'), (27, 'аы'), (28, 'аэ'), (29, 'аю'), (30, 'ая'), (31, 'ба'), (32, 'бб'), (33, 'бв'), (34, 'бг'), (35, 'бд'), (36, 'бе'), (37, 'бж'), (38, 'бз'), (39, 'би'), (40, 'бй'), (41, 'бк'), (42, 'бл'), (43, 'бм'), (44, 'бн'), (45, 'бо'), (46, 'бп'), (47, 'бр'), (48, 'бс'), (49, 'бт'), (50, 'бу'), (51, 'бф'), (52, 'бх'), (53, 'бц'), (54, 'бч'), (55, 'бш'), (56, 'бщ'), (57, 'бь'), (58, 'бы'), (59, 'бэ'), (60, 'бю'), (61, 'бя'), (62, 'ва'), (63, 'вб'), (64, 'вв'), (65, 'вг'), (66, 'вд'), (67, 'ве'), (68, 'вж'), (69, 'вз'), (70, 'ви'), (71, 'вй'), (72, 'вк'), (73, 'вл'), (74, 'вм'), (75, 'вн'), (76, 'во'), (77, 'вп'), (78, 'вр'), (79, 'вс'), (80, 'вт'), (81, 'ву'), (82, 'вф'), (8

In [27]:
big_dict[25]

'ащ'

In [28]:
big_dict[837]

'ыа'


End of test

Finding bigram codes of most frequent bigrams in plaintext(X_stars) and ciphertext(Y_stars)


In [29]:
X_stars = [ngram_code(ngram) for ngram in top_ngrams_rus]
X_stars

[545, 417, 572, 403, 168]

In [30]:
Y_stars = [ngram_code(ngram) for ngram in top_ngrams]
Y_stars

[20, 950, 153, 15, 423]

Function to find keys (a, b) for system of 2 modular equations

In [31]:
def key_finder(x1, x2, y1, y2):
    '''Using math functions to dolve modular equations to find keys (a, b)'''
    a = equation_solver(x1-x2, y1-y2, m**2)
    b_ = [equation_solver(1, y1-(a_*x1),  m**2) for a_ in a]
    b = [item for sublist in b_ for item in sublist]
    return a, b

In [32]:
test_keys = key_finder(X_stars[0], X_stars[1], Y_stars[0], Y_stars[1])
test_keys

([248], [361])

In [33]:
def decode(a, b):
    result = ''
    data = [text[i:i+2] for i in range(0,(len(text)-1),2)]
    for ngram in data:
        y = ngram_code(ngram)
        x = equation_solver(a, y-b, m**2)
        #x = (modinv(a, m**2)  * (y-b) ) % (m**2)
        result+=big_dict[x[0]]
    return result

In [34]:
test_result = decode(*test_keys[0], *test_keys[1])
test_result

ModuloError: No solutions

Function that marks only those texts that are possible for russian language

In [35]:
def check_text(data, a, b, ranges):
    '''Checking frequencies of letters 'ф' and 'э' of all ciphertexts and leaves only those that have frequencies <= 0.01, 
    because real frequencies of 'ф' and 'э' and are much smaller than 0.01'''
    global success
    n = len(data)
    counter = Counter(data)
    freq = {k: v/n for k, v in counter.items()}
    if ((freq['ф'] > 0.01) or (freq['э'] > 0.01)) :
        print('Not the right key')
        pass
    else:
        success +=1
        print('Successful variant # {}'.format(success))
        with open('success/result_{}'.format(success), 'w') as outfile:
            outfile.write(data)
            
        print('a={}, b={}, ranges = {} '.format(a, b, ranges), data[:100])

Global variable that controls the name of the file that is written to folder 'success'

In [36]:
success=0

In [37]:
len(ranges)

20

Finding all possible keys a and b, decrypting ciphrotext with them and checking whether these texts are possible in russian language

In [38]:
def dechifer():
    for x1, x2 in ranges:
        for y1, y2 in ranges:
            try:
                    a, b = key_finder(X_stars[x1], X_stars[x2], Y_stars[y1], Y_stars[y2])
                    for i in range(len(a)):
                            result = decode(a[i], b[i])
                            check_text(result, a[i], b[i], ((x1, x2), (y1, y2)))
            except ModuloError:
                    continue

In [39]:
dechifer()

Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the right key
Not the ri

Checking our resulting text

In [40]:
open('success/result_1', 'r').read()

'раннеераннееутропервыеотсветызаринакрышезаокномвселистьянадеревьяхвздрагиваютотзываясьнамалейшеедуновениепредрассветноговетеркаивотгдетодалекоиззаповоротанасеребряныхрельсахпоявляетсятрамвайпокачиваясьначетырехмаленькихсероголубыхколесахяркооранжевыйкакмандариннанемэполетымерцающеймедиизолотойкантпроводовижелтыйзвонокгромкозвякаетедвадопотопныйвожатыйстукнетпонемуногойвстоптанномбашмакецифрынабокахтрамваяиспередияркозолотыекаклимонсиденьяточнопорослипрохладнымзеленыммхомнакрышесловнозанесеногромныйкучерскойбичнабегуонскользитпосеребрянойпаутинепротянутойвысокосредидеревьевизвсехоконбудтоладаномпахнетвсепроникающимголубымизагадочнымзапахомлетнихгрозимолнийтрамвайзвенитвдольокаймленныхвязамиулициобтянутаясеройперчаткойрукавожатогоопятьиопятьлегкокасаетсярукоятоквполденьвожатыйостановилвагонпосредикварталаивысунулсявокошкоэйизавидевпризывныйвзмахсеройперчаткидугласчарлитомвсемальчишкиидевчонкивсегокварталакубаремскатилисьсдеревьевпобросаливтравускакалкионитакиосталисьлежатьсловнобелыезме

# Playground to play with different ciphertexts

Putting it all together

Change the text variable to the path of ciphertext

In [41]:
#text = open('for_test/V2', 'r').read()
text = open('variants/9.txt', 'r').read()
text = text.replace('\n', '')
text[:100]

FileNotFoundError: [Errno 2] No such file or directory: 'variants/9.txt'

In [ ]:
success = 0

In [ ]:
num_ngrams = ngram_counter(text, return_freq=True)
top_ngrams = sorted(num_ngrams.items(), key = lambda x: x[1], reverse=True)[:5]
top_ngrams = [x[0] for x in top_ngrams]
Y_stars = [ngram_code(ngram) for ngram in top_ngrams]
X_stars = [ngram_code(ngram) for ngram in top_ngrams_rus]
dechifer()